# Benchmark DRL with Q-Network

```BibTeX
@article{theate2021application,
  title={An application of deep reinforcement learning to algorithmic trading},
  author={Th{\'e}ate, Thibaut and Ernst, Damien},
  journal={Expert Systems with Applications},
  volume={173},
  pages={114632},
  year={2021},
  publisher={Elsevier}
}
```

## Setup Notebook

In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
import sys
warnings.filterwarnings("ignore")

INSTALL_DEPS = False
if INSTALL_DEPS:
    # If Kaggle or Colab, you have to manage these. If local, install requirements.txt
    %pip install yfinance==0.2.43
    # %pip install scipy==1.14.1
    # %pip install statsmodels==0.14.2
    # %pip install tabulate==0.9.0
    # %pip install torch==2.4.1
    # %pip install tqdm==4.66.5
    # %pip install gym==0.26.1
    # %pip install numpy==2.1.1
    # %pip install pandas==2.2.2

IN_KAGGLE = IN_COLAB = False
try:
    # https://www.tensorflow.org/install/pip#windows-wsl2
    import google.colab
    from google.colab import drive

    drive.mount("/content/drive")
    DATA_PATH = "/drive/input/drl-dataset-quant"
    IN_COLAB = True
    print("Colab!")
except:
    IN_COLAB = False
if "KAGGLE_KERNEL_RUN_TYPE" in os.environ and not IN_COLAB:
    print("Running in Kaggle...")
    for dirname, _, filenames in os.walk("/kaggle/input"):
        for filename in filenames:
            print(os.path.join(dirname, filename))
    DATA_PATH = "/kaggle/input/drl-dataset-quant"
    sys.path.insert(1, "/kaggle/usr/lib/drlutil")
    IN_KAGGLE = True
    print("Kaggle!")
elif not IN_COLAB:
    IN_KAGGLE = False
    DATA_PATH = "./data/"
    print("running localhost!")

In [2]:
from drlutil import TradingSimulator

## Experiment Parameters

In [ ]:
simulator = TradingSimulator()
strategy = "TDQN"
stock = "Tesla"

features=['Close', 'Low', 'High', 'Volume']
money=100000.
stateLength = 30
observationSpace = 1 + ((stateLength- 1)*len(features))
actionSpace = 2
bounds = [1, 30]
step = 1
numberOfEpisodes = 50
startingDate = '2012-1-1'
endingDate = '2020-1-1'
splitingDate = '2018-1-1'
percentageCosts = [0, 0.1, 0.2]
transactionCosts = percentageCosts[1]/100


print(observationSpace)

# Run Experiments

## Test Selected Stock and Strategy

In [ ]:

simulator.simulateNewStrategy(strategy,
                              stock,
                              startingDate=startingDate,
                              endingDate=endingDate,
                              splitingDate=splitingDate,
                              rendering=True,
                              saveStrategy=False,
                              data_dir=DATA_PATH,
                              money=money,
                              observationSpace=observationSpace,
                              actionSpace=actionSpace,
                              stateLength=stateLength,
                              bounds=bounds,
                              step=step,
                              numberOfEpisodes=numberOfEpisodes,
                              transactionCosts=transactionCosts,
                              features=features)


# Experiment Results

| Performance Indicator           | Training  | Testing  | B&H      | S&H      | TF      | MR      | TDQN    |
|----------------------------------|-----------|----------|----------|----------|---------|---------|---------|
| Profit & Loss (P&L)              | 11686926  | -9801    | 29847    | -29847   | 73301   | 8600    | 98      |
| Annualized Return                | 36.25%    | 8.03%    | 24.11%   | -7.38%   | 100.00% | 19.02%  | 12.80%  |
| Annualized Volatility            | 45.32%    | 51.64%   | 53.14%   | 46.11%   | 52.70%  | 58.05%  | 52.09%  |
| Sharpe Ratio                     | 1.983     | 0.162    | 0.508    | -0.154   | 0.987   | 0.358   | 0.261   |
| Sortino Ratio                    | 3.196     | 0.204    | 0.741    | -0.205   | 1.229   | 0.539   | 0.359   |
| Maximum Drawdown                 | 15.31%    | 60.36%   | 52.83%   | 54.09%   | 79.91%  | 65.31%  | 58.95%  |
| Maximum Drawdown Duration        | 47 days   | 95 days  | 205 days | 144 days | 229 days| 159 days| 331 days|
| Profitability                    | 54.82%    | 46.38%   | 100.00%  | 0.00%    | 34.38%  | 67.65%  | 38.18%  |
| Ratio Average Profit/Loss        | 3.082     | 1.112    | ∞        | 0.00     | 0.534   | 0.496   | 1.621   |

Training time: 19:08mins
